In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reading the DATA SET


In this notebook, I developed a model aimed at predicting flight delays at the Destination Airport. The purpose is to create a Data set which can be used for visualization and model Building so as to predict the Delays of Flights. I did the visualization so as to get better inferences about the data. For, model fitting I have seperated the Dataset into training Data and Testing Data so that prediction can be done on the testing Data. I also showed how to import Tableau and make visualization more crisp and clear.

Technical aspect Covered:

visualization: matplolib, seaborn, Tableau

data manipulation: pandas, numpy

modeling: sklearn

class definition: regression, Boosting, Bagging

For EDA I used some part of Python coding and Tableau Visulization so as to get a brief insight and inference from the data. Various Plots are created so as to get a great idea of whats happening in the Dataset and what is the most important variable affecting the dalays of the airlines. Feature scaling is a method used to normalize the range of independent variables or features of data and this concept is used.



In [2]:
data = pd.read_csv("archive/flights.csv")
airport = pd.read_csv('archive/airports.csv')
airlines = pd.read_csv('archive/airlines.csv')

/var/folders/2w/hy3qhkf90ld86v626flq0kb80000gn/T/ipykernel_6364/3981750233.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("archive/flights.csv")


In [3]:
# data.info()

In [4]:
# data.shape

The Data Contains 31 columns and 1048575 Rows

In [5]:
# data.describe()

In [6]:
# data.head()

In [7]:
# data.tail()

In [8]:
# data

From the above table it is clear that data is not properly organised and date is given seperated and many columns have unnecessary data not useful for visualization for which it is required that we clean the data and take only those columns which is of our use.

In [9]:
# airport.isnull().sum()

In [10]:
airport = airport.dropna(subset = ['LATITUDE','LONGITUDE'])

In [11]:
# airport.isnull().sum()

In [12]:
# airport.head(10)

In [13]:
# airlines

In [14]:
# Data_NULL = data.isnull().sum()*100/data.shape[0]
# Data_NULL

We can see that 96% of the values in Cancellation reason column are null for which it is of less use while predicting Delays. Some other columns include 78.2% in Air System Delay, Security Delay, Airline Delay, Weather Delay etc. So I am going to create two Dataset which is having no null values one is by removing all the null values irrespective of different types of Delays and other I am going to take the data set with respect to different types of delays. The first Dataset is named as Flights and the other one is named as Flight_Delays.

In [15]:
data[['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']] = data[['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']].fillna(0)

In [16]:
# # Dropping of subset of null values
data1 = data.dropna(subset = ["TAIL_NUMBER",'DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME',
             'ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [17]:
data1.shape

(5714008, 31)

In [18]:
# data1.isnull().sum()

In [19]:
# Creting Dataset w.r.t different Types of Delays
data11 = data1.dropna(subset = ['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
# data11 = data1[['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']]
data11 = data11.drop(['YEAR','MONTH','DAY','DAY_OF_WEEK','TAIL_NUMBER','SCHEDULED_DEPARTURE','DEPARTURE_TIME','SCHEDULED_TIME',
                     'SCHEDULED_ARRIVAL','ARRIVAL_TIME','DIVERTED','CANCELLED','CANCELLATION_REASON','FLIGHT_NUMBER','WHEELS_OFF',
                     'WHEELS_ON','AIR_TIME'],axis = 1)

In [20]:
# data11.info()

In [21]:
data11.columns

Index(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY',
       'TAXI_OUT', 'ELAPSED_TIME', 'DISTANCE', 'TAXI_IN', 'ARRIVAL_DELAY',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [22]:
# The other Dataset
Flight_Delays = data11

In [23]:
# Creating Dataset by removing null values by not focussing fully on different types of Delays
data2 = data1.drop(['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY',
                    'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'],axis = 1)

In [24]:
# data2.isnull().sum()

In [25]:
data2.shape

(5714008, 25)

In [26]:
# data2.info()

In [27]:
# data2.DEPARTURE_TIME.isnull().sum()

In [28]:
# data2.DEPARTURE_TIME.dtype

In [29]:
# data2.DEPARTURE_TIME

The departure time above is not very much informative so we are going to change it in the datetime format so that we get a better idea of the time.

In [30]:
# Creating a function to change the way of representation of time in the column
def Format_Hourmin(hours):
        if hours == 2400:
            hours = 0
        else:
            hours = "{0:04d}".format(int(hours))
            Hourmin = datetime.time(int(hours[0:2]), int(hours[2:4]))
            return Hourmin

In [31]:
data2['Actual_Departure'] =data1['DEPARTURE_TIME'].apply(Format_Hourmin)
data2['Actual_Departure']

0          23:54:00
1          00:02:00
2          00:18:00
3          00:15:00
4          00:24:00
             ...   
5819074    23:55:00
5819075    23:55:00
5819076    23:50:00
5819077    23:53:00
5819078    00:14:00
Name: Actual_Departure, Length: 5714008, dtype: object

In [32]:
# data2.columns

In [33]:
# Creating Date in the Datetime format
data2['Date'] = pd.to_datetime(data2[['YEAR','MONTH','DAY']])
# data2.Date

In [34]:
data2['Day'] = data2['Date'].dt.day_name()
# data2['Day']

In [35]:
# Applying the function to required variables in the dataset
data2['Actual_Departure'] =data1['DEPARTURE_TIME'].apply(Format_Hourmin)
data2['Scheduled_Arrival'] =data1['SCHEDULED_ARRIVAL'].apply(Format_Hourmin)
data2['Scheduled_Departure'] =data1['SCHEDULED_DEPARTURE'].apply(Format_Hourmin)
data2['Actual_Arrival'] =data1['ARRIVAL_TIME'].apply(Format_Hourmin)

# Merging of  3 data sets

Since there are three dataset it is required to merge all the three data set so that we can use it during the visualization in a proper way.

In [36]:
# Merging on AIRLINE and IATA_CODE
data2 = data2.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='inner')

In [37]:
data2 = data2.drop(['AIRLINE_x','IATA_CODE'], axis=1)

In [38]:
data2 = data2.rename(columns={"AIRLINE_y":"AIRLINE"})

In [39]:
data2 = data2.merge(airport, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='inner')
data2 = data2.merge(airport, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='inner')

In [40]:
# data2.columns

In [41]:
data2 = data2.drop(['LATITUDE_x', 'LONGITUDE_x',
       'STATE_y', 'COUNTRY_y', 'LATITUDE_y', 'LONGITUDE_y','STATE_x', 'COUNTRY_x'], axis=1)

In [42]:
data2 = data2.rename(columns={'IATA_CODE_x':'Org_Airport_Code','AIRPORT_x':'Org_Airport_Name','CITY_x':'Origin_city',
                             'IATA_CODE_y':'Dest_Airport_Code','AIRPORT_y':'Dest_Airport_Name','CITY_y':'Destination_city'})

In [43]:
# data2

In [44]:
# data11.columns

In [45]:
# data2.columns

In [46]:
# we are taking the required data into Account for visualization and the Analysis
ReqdData = pd.DataFrame(data2[['AIRLINE','Org_Airport_Name','Origin_city',
                               'Dest_Airport_Name','Destination_city','ORIGIN_AIRPORT',
                               'DESTINATION_AIRPORT','DISTANCE','Actual_Departure','Date','Day',
                               'Scheduled_Departure','DEPARTURE_DELAY','Actual_Arrival','Scheduled_Arrival','ARRIVAL_DELAY',
                              'SCHEDULED_TIME','ELAPSED_TIME','AIR_TIME','TAXI_IN','TAXI_OUT','DIVERTED',]])

In [47]:
# data2.DEPARTURE_TIME.dtype

In [48]:
ReqdData = ReqdData.dropna(subset = ['Actual_Departure','Actual_Arrival'])

In [49]:
# ReqdData.info()

In [50]:
# Cleaned Dataset for visualization and Analysis
Flights = ReqdData

In [51]:
# plt.figure(figsize=(10, 10))
# axis = sns.countplot(x=Flights['Origin_city'], data = Flights,
#               order=Flights['Origin_city'].value_counts().iloc[:20].index)
# axis.set_xticklabels(axis.get_xticklabels(), rotation=90, ha="right")
# plt.tight_layout()
# plt.show()

# The Figure shows that Atlanta has the highest count of flight from origin city 

In [52]:
# plt.figure(figsize=(10, 10))
# axis = sns.countplot(x=Flights['Org_Airport_Name'], data = Flights,
#               order=Flights['Org_Airport_Name'].value_counts().iloc[:20].index)
# axis.set_xticklabels(axis.get_xticklabels(), rotation=90, ha="right")
# plt.tight_layout()
# plt.show()

In [53]:
# axis = plt.subplots(figsize=(10,14))
# Name = Flights["AIRLINE"].unique()
# size = Flights["AIRLINE"].value_counts()
# plt.pie(size,labels=Name,autopct='%5.0f%%')
# plt.show()

In [54]:
# axis = plt.subplots(figsize=(10,14))
# sns.despine(bottom=True, left=True)
# # Observations with Scatter Plot
# sns.stripplot(x="ARRIVAL_DELAY", y="AIRLINE",
#               data = Flights, dodge=True, jitter=True
#             )
# plt.show()

American Airlines Inc has the highest Arrival Delay.

# Prediction

In [55]:
# Flights.head()

In [56]:
# Flights.columns

In [57]:
# # Dropping unncecssary columns before prediction
Flights1 = Flights.drop(['Org_Airport_Name','Origin_city','Dest_Airport_Name','Destination_city'],axis = 1)

In [58]:
Flights1.columns

Index(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE',
       'Actual_Departure', 'Date', 'Day', 'Scheduled_Departure',
       'DEPARTURE_DELAY', 'Actual_Arrival', 'Scheduled_Arrival',
       'ARRIVAL_DELAY', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME',
       'TAXI_IN', 'TAXI_OUT', 'DIVERTED'],
      dtype='object')

In [59]:
# # To check the Distribution of Air Time
# sns.displot(Flights1['AIR_TIME'])
# plt.show()

In [60]:
# # To check the Distribution of Elapsed Time
# sns.displot(Flights1['ELAPSED_TIME'])
# plt.show()

In [61]:
# # To check the Distribution of Taxi IN
# sns.displot(Flights1['TAXI_IN'])
# plt.show()

In [62]:
# # To check the Distribution of Taxi out
# sns.displot(Flights1['TAXI_OUT'])
# plt.show()

In [63]:
# importing Various regression algorithms 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso,LinearRegression,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [64]:
Las = Lasso()
LinR = LinearRegression()
Rid = Ridge()
Rfc = RandomForestRegressor(random_state=2)
Dtc = DecisionTreeRegressor(random_state = 2)
Boost_Lin = AdaBoostRegressor(estimator=LinR,random_state=2)
Boost_las = AdaBoostRegressor(estimator=Las,random_state=2)
Boost_rid = AdaBoostRegressor(estimator=Rid,random_state=2)
Bg_Lin = BaggingRegressor(estimator=LinR,random_state=2)
Bg_las = BaggingRegressor(estimator=Las,random_state=2)
Bg_rid = BaggingRegressor(estimator=Rid,random_state=2)

In [65]:
le = LabelEncoder()

In [66]:
# Label encoding features to change categorical variables into numerical one
Flights1['AIRLINE']= le.fit_transform(Flights1['AIRLINE'])
Flights1['ORIGIN_AIRPORT'] = le.fit_transform(Flights1['ORIGIN_AIRPORT'])
Flights1['DESTINATION_AIRPORT'] = le.fit_transform(Flights1['DESTINATION_AIRPORT'])
Flights1['Day'] = le.fit_transform(Flights1['Day'])

In [67]:
Flights1 = Flights1.drop(['Scheduled_Departure','Scheduled_Arrival','Actual_Arrival','Date','Actual_Departure'], axis = 1)

In [68]:
X = Flights1.drop('ARRIVAL_DELAY',axis = 1)
# X.shape

In [69]:
y = Flights1['ARRIVAL_DELAY']
# y.head()

In [70]:
# Splitting into train and test data set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 2)

In [71]:
sc1=StandardScaler()
X_train_sc=sc1.fit_transform(X_train)
X_test_sc=sc1.transform(X_test)

# Model fitting and results

In [72]:
model1 = LinR.fit(X_train_sc,y_train)
Y1_predict=model1.predict(X_test_sc)
print("LinearRegression:")
print('Mean Absolute Error:', mean_absolute_error(y_test, Y1_predict))  
print('Mean Squared Error:', mean_squared_error(y_test, Y1_predict))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y1_predict)))
print('R2 : ',r2_score(y_test, Y1_predict))

LinearRegression:
Mean Absolute Error: 1.2773244090426017e-06
Mean Squared Error: 2.5546470946653774e-06
Root Mean Squared Error: 0.0015983263417291781
R2 :  0.999999998393613


In [73]:
# for model, name in zip([Las,LinR,Rid,Dtc,Rfc,Boost_Lin], 
#      ['Lasso','Linear Regression','Ridge','Decision Tree Regressor','Random forest Regressor','Boosted Linear']):
#     model1 = model.fit(X_train_sc,y_train)
#     Y_predict=model1.predict(X_test_sc)
#     print(name)
#     print('Mean Absolute Error:', mean_absolute_error(y_test, Y_predict))  
#     print('Mean Squared Error:', mean_squared_error(y_test, Y_predict))  
#     print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y_predict)))
#     print('R2 : ',r2_score(y_test, Y_predict))
#     print()

In [74]:
# for model, name in zip([Boost_las,Boost_rid,Bg_Lin,Bg_las,Bg_rid], 
#      ['Boosted Lasso','Boosted Ridge','Bagged Linear','Bagged Lasso','Bagged Ridge']):
#     model1 = model.fit(X_train_sc,y_train)
#     Y_predict=model1.predict(X_test_sc)
#     print(name)
#     print('Mean Absolute Error:', mean_absolute_error(y_test, Y_predict))  
#     print('Mean Squared Error:', mean_squared_error(y_test, Y_predict))  
#     print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y_predict)))
#     print('R2 : ',r2_score(y_test, Y_predict))
#     print()

# Model Analysis

In [75]:
# for model, name in zip([Las,LinR,Rid,Dtc,Rfc,Boost_Lin,Boost_las,Boost_rid,Bg_Lin,Bg_las,Bg_rid], 
#      ['Lasso','Linear Regression','Ridge','Random forest Regressor','Decision Tree Regressor','Boosted Linear',
#       'Boosted Lasso','Boosted Ridge','Bagged Linear','Bagged Lasso','Bagged Ridge']):
#     model1 = model.fit(X_train_sc,y_train)
#     Y_predict=model1.predict(X_test_sc)
#     print(name)
#     plt.scatter(y_test, Y_predict)
#     plt.title("Model Analysis")
#     plt.xlabel("Truth")
#     plt.ylabel("Prediction")
#     plt.show()

# When Flight_Delays is taken, where I took all the different Delays into Concern.

I removed the null values present in all the different types of Delays and proceeded with prediction of the Arrival Delays. 

In [76]:
# axis = plt.subplots(figsize=(10,14))
# Name = Flight_Delays["AIRLINE"].unique()
# size = Flight_Delays["AIRLINE"].value_counts()
# plt.pie(size,labels=Name,autopct='%5.0f%%')
# plt.show()

In [77]:
Flights1.columns

Index(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE', 'Day',
       'DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'SCHEDULED_TIME', 'ELAPSED_TIME',
       'AIR_TIME', 'TAXI_IN', 'TAXI_OUT', 'DIVERTED'],
      dtype='object')

In [78]:
Flight_Delays.columns

Index(['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY',
       'TAXI_OUT', 'ELAPSED_TIME', 'DISTANCE', 'TAXI_IN', 'ARRIVAL_DELAY',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [79]:
Flight_Delays['ORIGIN_AIRPORT'] = Flight_Delays['ORIGIN_AIRPORT'].astype(str)
Flight_Delays['DESTINATION_AIRPORT'] = Flight_Delays['DESTINATION_AIRPORT'].astype(str)

In [80]:
Flight_Delays['AIRLINE']= le.fit_transform(Flight_Delays['AIRLINE'])
Flight_Delays['ORIGIN_AIRPORT'] = le.fit_transform(Flight_Delays['ORIGIN_AIRPORT'])
Flight_Delays['DESTINATION_AIRPORT'] = le.fit_transform(Flight_Delays['DESTINATION_AIRPORT'])

In [81]:
X = Flight_Delays.drop('ARRIVAL_DELAY',axis = 1)
# X.shape
y = Flight_Delays['ARRIVAL_DELAY']
# y.head()

In [82]:
# Splitting the Data into Training and Testing set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 2)

In [83]:
# Scalling of the Data
sc1=StandardScaler()
X_train_sc=sc1.fit_transform(X_train)
X_test_sc=sc1.transform(X_test)

# Model fitting and results

In [84]:
model1 = Rid.fit(X_train_sc,y_train)
Y2_predict=model1.predict(X_test_sc)
print("Ridge Regression:")
print('Mean Absolute Error:', mean_absolute_error(y_test, Y2_predict))  
print('Mean Squared Error:', mean_squared_error(y_test, Y2_predict))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y2_predict)))
print('R2 : ',r2_score(y_test, Y2_predict))

Ridge Regression:
Mean Absolute Error: 6.199613900623898
Mean Squared Error: 65.09826201877826
Root Mean Squared Error: 8.068349398655108
R2 :  0.9573417498642477


In [85]:
# for model, name in zip([Las,LinR,Rid,Dtc,Rfc,Boost_Lin,Boost_las,Boost_rid,Bg_Lin,Bg_las,Bg_rid], 
#      ['Lasso','Linear Regression','Ridge','Random forest Regressor','Decision Tree Regressor','Boosted Linear',
#       'Boosted Lasso','Boosted Ridge','Bagged Linear','Bagged Lasso','Bagged Ridge']):
#     model1 = model.fit(X_train_sc,y_train)
#     Y_predict=model1.predict(X_test_sc)
#     print(name)
#     print('Mean Absolute Error:', mean_absolute_error(y_test, Y_predict))  
#     print('Mean Squared Error:', mean_squared_error(y_test, Y_predict))  
#     print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y_predict)))
#     print('R2 : ',r2_score(y_test, Y_predict))
#     print()

# Model Analysis

In [86]:
# for model, name in zip([Las,LinR,Rid,Dtc,Rfc,Boost_Lin,Boost_las,Boost_rid,Bg_Lin,Bg_las,Bg_rid], 
#      ['Lasso','Linear Regression','Ridge','Random forest Regressor','Decision Tree Regressor','Boosted Linear',
#       'Boosted Lasso','Boosted Ridge','Bagged Linear','Bagged Lasso','Bagged Ridge']):
#     model1 = model.fit(X_train_sc,y_train)
#     Y_predict=model1.predict(X_test_sc)
#     print(name)
#     plt.scatter(y_test, Y_predict)
#     plt.title("Model Analysis")
#     plt.xlabel("Truth")
#     plt.ylabel("Prediction")
#     plt.show()

# Conclusion

We can see that maximum arrival Delays are dependent on the Departure Delays of the Airport. The first part is dealt with cleaning and exploration of the data set to get more insights and the second part dealt with the sitting of the model to predict the delays. For exploratory Analysis I used visualization tools like tableau and seaborn as well as matplotlib. The Second part dealt with the model fitting and predicting the Arrival delays of the airlines.

We can see that departure delay is the main problem which is creating Delay in the aviation industry. Departure Delays can be caused due Security Delay, Airline System Delays, Airlines Delay etc. The Delays affect the revenue of the company to a great extent so the delays has to be reduced as much as possible so as to increase the profitability in the Airline Industry. Customer Satisfaction will also be greatly enhanced if the delays can be brought down as low as possible.

# Choosing one model per dataset

Since we want just one per dataset (not 15 models), here’s what makes sense:

Without Delays Dataset → Linear Regression 

Linear Regression already explains almost all variance with negligible error.

Bagging doesn’t add much, so plain Linear Regression is simple & effective.

With Delays Dataset → Ridge Regression 

Linear Regression also performs perfectly here, but Ridge adds stability against collinearity and small noise.

Since your dataset may have correlated features, Ridge would be slightly safer long-term.

# Average

# Weighted Average

# Meta model